# Project: Predicting Diabetes Onset FHIR EHR Data
### Notebook: 02_Data_Cleaning.ipynb
### Purpose: Clean raw data

In [133]:
import pandas as pd
import numpy as np
from typing import List, Tuple
from pathlib import Path

In [135]:
# Helper function to load data
def load_data(file_names: List[str], path: str) -> List[pd.DataFrame]:
    dfs = []
    base_path = Path(path)
    
    for name in file_names:
        file_path = base_path / name

        try:
            df = pd.read_csv(file_path)
            dfs.append(df)
        except fileNotFoundError:
            print(f"[WARNING] File not found at: {file_path}")
        except pd.errors.EmptyDataError:
            print(f"[WARNING] File empty: {file_path}")
        except Exception as e:
            print(f"Failed to load data at {file_path}:{e}")
    return dfs

In [137]:
# Load data
file_names = ['Conditions_Raw.csv',
             'Patients_Raw.csv',
             'Encounters_Raw.csv',
             'Observations_Raw.csv']
base_path = '/Users/sanasiddiqui/Desktop/Current Desktop/Data Portfolio Projects/Diabetes_Onset_Prediction_FHIR/Data_Diabetes_Prediction/Raw'
dfs = load_data(file_names, path=base_path)


In [139]:
df_conditions_raw = dfs[0]
df_patients_raw = dfs[1]
df_encounters_raw = dfs[2]
df_observations_raw = dfs[3]

## Preliminary Data Validation of Response Variable (Diabetes)

In [141]:
# check data frame shapes
print(
    f"Patients: {df_patients_raw.shape}\n" 
    f"Observations: {df_observations_raw.shape}\n" 
    f"Encounters: {df_encounters_raw.shape}\n" 
    f"Conditions: {df_conditions_raw.shape}")

Patients: (1159, 3)
Observations: (585691, 6)
Encounters: (66166, 5)
Conditions: (41676, 7)


In [143]:
# Create a flag for prediabetic patients to do initial validation
# confirming if we have enough prediabetic and diabetic patients

df_conditions_raw["is_prediabetes"] = df_conditions_raw["name"].str.lower().str.contains("prediabetes")

In [145]:
# Create a flag for diabetic patients to do initial validation
# confirming if we have enough prediabetic and diabetic patients

df_conditions_raw["is_diabetes"] = df_conditions_raw["name"].str.lower().str.contains("diabetes")

In [147]:
# count prediabetic patients
n_prediabetic_pts = df_conditions_raw[df_conditions_raw["is_prediabetes"]]["patient_id"].nunique()
n_prediabetic_pts

445

In [151]:
# count diabetic patients
n_diabetic_pts = df_conditions_raw[df_conditions_raw["is_diabetes"]]["patient_id"].nunique()
n_diabetic_pts

503

In [96]:
# convert onset column into datetime object
df_conditions_raw['onset'] = pd.to_datetime(df_conditions_raw['onset'], errors='coerce')

/var/folders/jr/vyvz729175v013k_g7gzk_7c0000gn/T/ipykernel_68474/2443224953.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_conditions_raw['onset'] = pd.to_datetime(df_conditions_raw['onset'], errors='coerce')


In [153]:
# find all pre and diabetic patients,
# merge rows to find how many prediabetic pts converting to diabetic
pre = df_conditions_raw[df_conditions_raw['is_prediabetes']]
dm = df_conditions_raw[df_conditions_raw['is_diabetes']]

# merge
merged = pre.merge(
    dm,
    on='patient_id',
    suffixes=('_pre','_dm')
)


In [155]:
# find patients progressing from prediabetes to diabetes

progressors = merged[merged['onset_pre'] < merged['onset_dm']]
progressors.head()

,patient_id,condition_id_pre,code_pre,name_pre,onset_pre,recorded_date_pre,clinical_status_pre,is_prediabetes_pre,is_diabetes_pre,condition_id_dm,code_dm,name_dm,onset_dm,recorded_date_dm,clinical_status_dm,is_prediabetes_dm,is_diabetes_dm
5,urn:uuid:dbc13ff9-face-3a16-b7b2-0073d73dea9a,dbc13ff9-face-3a16-3595-9b1b323e77e9,714628002,Prediabetes (finding),1987-02-27T12:12:34-05:00,1987-02-27T12:12:34-05:00,active,True,True,dbc13ff9-face-3a16-1429-2dce430cf0a7,127013003,Disorder of kidney due to diabetes mellitus (d...,2015-03-27T13:12:34-04:00,2015-03-27T13:12:34-04:00,active,False,True
6,urn:uuid:dbc13ff9-face-3a16-b7b2-0073d73dea9a,dbc13ff9-face-3a16-3595-9b1b323e77e9,714628002,Prediabetes (finding),1987-02-27T12:12:34-05:00,1987-02-27T12:12:34-05:00,active,True,True,dbc13ff9-face-3a16-5bac-b481539326a1,90781000119102,Microalbuminuria due to type 2 diabetes mellit...,2018-04-13T13:12:34-04:00,2018-04-13T13:12:34-04:00,active,False,True
7,urn:uuid:dbc13ff9-face-3a16-b7b2-0073d73dea9a,dbc13ff9-face-3a16-3595-9b1b323e77e9,714628002,Prediabetes (finding),1987-02-27T12:12:34-05:00,1987-02-27T12:12:34-05:00,active,True,True,dbc13ff9-face-3a16-2bbd-5dbdc71ab9c4,157141000119108,Proteinuria due to type 2 diabetes mellitus (d...,2019-01-04T12:12:34-05:00,2019-01-04T12:12:34-05:00,active,False,True
9,urn:uuid:3e5092da-7854-5cfe-2cfb-8f3c96e182fd,3e5092da-7854-5cfe-4033-d2939757e402,714628002,Prediabetes (finding),1971-05-02T15:00:43-04:00,1971-05-02T15:00:43-04:00,active,True,True,3e5092da-7854-5cfe-1df2-b3c726cc5f7d,127013003,Disorder of kidney due to diabetes mellitus (d...,2008-04-06T15:00:43-04:00,2008-04-06T15:00:43-04:00,active,False,True
10,urn:uuid:3e5092da-7854-5cfe-2cfb-8f3c96e182fd,3e5092da-7854-5cfe-4033-d2939757e402,714628002,Prediabetes (finding),1971-05-02T15:00:43-04:00,1971-05-02T15:00:43-04:00,active,True,True,3e5092da-7854-5cfe-54ff-f12667c85139,90781000119102,Microalbuminuria due to type 2 diabetes mellit...,2011-07-03T15:00:43-04:00,2011-07-03T15:00:43-04:00,active,False,True


In [157]:
progressors.shape

(263, 17)

## Data Cleaning

In [159]:
# copy data to preserve raw data

df_patients = df_patients_raw.copy()
df_obs = df_observations_raw.copy()
df_enc = df_encounters_raw.copy()
df_cond = df_conditions_raw.copy()

#### Structural Checks

In [103]:
# Helper function to inspect dataframe structure

def inspect_structure(df, name: str, n: int = 5) -> None:
    """inspect structure, columns, data types, and first n rows
    of dataframe

    Parameters:
    df: dataframe
    name: name of dataframe
    n: number of first few rows in dataframe
    """
    print(f"\n=== {name} ===")
    print(f"\nShape: {df.shape}")
    print("\nColumns: ")
    print(list(df.columns))
    print("\nDtypes: ")
    print(df.dtypes)
    print(f"\nPreview (first {n} rows):")
    print(df.head(n))

In [161]:
# Wrapper function to check structure of multiple dataframes at once

def run_structural_checks(dfs: dict) -> None:
    for name, df in dfs.items():
        inspect_structure(df, name)

In [163]:
# dictionary holding all dataframes and their names

dfs = {
    "Patients": df_patients,
    "Observations": df_obs,
    "Encounters": df_enc,
    "Conditions": df_cond
}

In [165]:
# inspect all dataframe structures

run_structural_checks(dfs)


=== Patients ===

Shape: (1159, 3)

Columns: 
['patient_id', 'gender', 'birth_date']

Dtypes: 
patient_id    object
gender        object
birth_date    object
dtype: object

Preview (first 5 rows):
                             patient_id  gender  birth_date
0  a8a4c7d6-722a-a914-04b6-a23a7c3496e2  female  1999-05-21
1  779b82f4-c8cd-9764-2f80-28f0b176a418  female  2000-03-03
2  e696153b-4bda-a741-04fe-a1fd43f60fd9  female  1972-04-14
3  7e0366e0-5cf1-1f51-f609-3ec9c2197140  female  1998-03-26
4  f286e228-2d19-4405-da1c-23dcae75169b  female  1971-04-03

=== Observations ===

Shape: (585691, 6)

Columns: 
['patient_id', 'code', 'name', 'value', 'unit', 'time']

Dtypes: 
patient_id     object
code           object
name           object
value         float64
unit           object
time           object
dtype: object

Preview (first 5 rows):
                                      patient_id    code  \
0  urn:uuid:a8a4c7d6-722a-a914-04b6-a23a7c3496e2  6206-7   
1  urn:uuid:a8a4c7d6-722a-a914-0

In [167]:
# ensure no NA patient ids and all present in df_patients

assert df_obs['patient_id'].notna().all()
assert df_enc['patient_id'].notna().all()
assert df_cond['patient_id'].notna().all()

In [108]:
assert df_obs['patient_id'].isin(df_patients['patient_id']).all()

AssertionError: 

In [169]:
invalid_obs = df_obs.loc[
    ~df_obs['patient_id'].isin(df_patients['patient_id'])
]
len(invalid_obs)

585691

In [171]:
df_patients.shape, df_obs.shape

((1159, 3), (585691, 6))

In [173]:
df_patients.columns, df_obs.columns

(Index(['patient_id', 'gender', 'birth_date'], dtype='object'),
 Index(['patient_id', 'code', 'name', 'value', 'unit', 'time'], dtype='object'))

In [175]:
df_patients.head(), df_obs.head(), df_enc.head(), df_cond.head()

(                             patient_id  gender  birth_date
 0  a8a4c7d6-722a-a914-04b6-a23a7c3496e2  female  1999-05-21
 1  779b82f4-c8cd-9764-2f80-28f0b176a418  female  2000-03-03
 2  e696153b-4bda-a741-04fe-a1fd43f60fd9  female  1972-04-14
 3  7e0366e0-5cf1-1f51-f609-3ec9c2197140  female  1998-03-26
 4  f286e228-2d19-4405-da1c-23dcae75169b  female  1971-04-03,
                                       patient_id    code  \
 0  urn:uuid:a8a4c7d6-722a-a914-04b6-a23a7c3496e2  6206-7   
 1  urn:uuid:a8a4c7d6-722a-a914-04b6-a23a7c3496e2  6273-7   
 2  urn:uuid:a8a4c7d6-722a-a914-04b6-a23a7c3496e2  6082-2   
 3  urn:uuid:a8a4c7d6-722a-a914-04b6-a23a7c3496e2  6246-3   
 4  urn:uuid:a8a4c7d6-722a-a914-04b6-a23a7c3496e2  6276-0   
 
                                      name     value  unit  \
 0   Peanut IgE Ab [Units/volume] in Serum  0.088943  kU/L   
 1   Walnut IgE Ab [Units/volume] in Serum  0.086704  kU/L   
 2  Codfish IgE Ab [Units/volume] in Serum  0.033234  kU/L   
 3   Shrimp IgE A

In [181]:
# helper function to format patient_id in all dataframes exactly as seen in df_patients

def strip_urn_uuid(x):
    """remove urn:uuid: prefix from patient_id
    from all dataframes
    """
    
    if pd.isna(x):
        return None
    x = str(x)
    return x.replace("urn:uuid:", "")

In [183]:
# modify patient_id format in all dataframes to match patient_id in df_patients

for df in [df_obs, df_enc, df_cond]:
    df['patient_id'] = df['patient_id'].apply(strip_urn_uuid)

In [185]:
# validate all patient_ids actoss all dataframes match with those in df_patients

assert df_obs['patient_id'].isin(df_patients['patient_id']).all()
assert df_enc['patient_id'].isin(df_patients['patient_id']).all()
assert df_cond['patient_id'].isin(df_patients['patient_id']).all()

#### Data Types and Conversions

In [187]:
# df_obs time column converted to datetime

df_obs['time'] = pd.to_datetime(df_obs['time'], utc=True)
df_obs.dtypes

patient_id                 object
code                       object
name                       object
value                     float64
unit                       object
time          datetime64[ns, UTC]
dtype: object

In [189]:
for col in ['start','end']:
    df_enc[col] = pd.to_datetime(df_enc[col], utc=True)
df_enc.dtypes

patient_id                   object
encounter_id                 object
start           datetime64[ns, UTC]
end             datetime64[ns, UTC]
class                        object
dtype: object

In [191]:
for col in ['onset', 'recorded_date']:
    df_cond[col] = pd.to_datetime(df_cond[col], utc=True)
df_cond.dtypes

patient_id                      object
condition_id                    object
code                             int64
name                            object
onset              datetime64[ns, UTC]
recorded_date      datetime64[ns, UTC]
clinical_status                 object
is_prediabetes                    bool
is_diabetes                       bool
dtype: object

#### Missing Data

In [193]:
df_obs.isna().mean().sort_values(ascending=False)

value         0.22237
unit          0.22237
patient_id    0.00000
code          0.00000
name          0.00000
time          0.00000
dtype: float64

In [195]:
(
    df_obs[df_obs['value'].isna()]
    .groupby('name')
    .size()
    .sort_values(ascending=False)
    .head(15)
)
    

name
Blood pressure panel with all children optional                                                16430
Tobacco smoking status                                                                         14815
Protocol for Responding to and Assessing Patients' Assets, Risks, and Experiences [PRAPARE]    11322
Ketones [Presence] in Urine by Test strip                                                       7262
Nitrite [Presence] in Urine by Test strip                                                       7262
Hemoglobin [Presence] in Urine by Test strip                                                    7262
Leukocyte esterase [Presence] in Urine by Test strip                                            7262
Appearance of Urine                                                                             7142
Color of Urine                                                                                  7091
Glucose [Presence] in Urine by Test strip                                             

In [197]:
#filter rows with numeric data only

df_obs_numeric = df_obs.loc[df_obs['value'].notna() & df_obs['unit'].notna()]
df_obs_numeric.isna().mean()

patient_id    0.0
code          0.0
name          0.0
value         0.0
unit          0.0
time          0.0
dtype: float64

In [199]:
df_enc.isna().mean().sort_values(ascending=False)

patient_id      0.0
encounter_id    0.0
start           0.0
end             0.0
class           0.0
dtype: float64

In [201]:
df_cond.isna().mean().sort_values(ascending=False)

patient_id         0.0
condition_id       0.0
code               0.0
name               0.0
onset              0.0
recorded_date      0.0
clinical_status    0.0
is_prediabetes     0.0
is_diabetes        0.0
dtype: float64

In [203]:
# Helper function to find duplicate rows

def find_duplicates(dfs: dict):
    for name, df in dfs.items():
        print(name)
        print(f"Duplicates Count: {df.duplicated().sum()}")

In [205]:
# find duplciates

dfs_updated =  {
    "Patients": df_patients,
    "Observations Numeric": df_obs_numeric,
    "Encounters": df_enc,
    "Conditions": df_cond
}
find_duplicates(dfs_updated)

Patients
Duplicates Count: 0
Observations Numeric
Duplicates Count: 234
Encounters
Duplicates Count: 0
Conditions
Duplicates Count: 0


In [207]:
# inspect observations df for duplicates

subset_cols = ['patient_id','code','time']
print(f"Duplicate observations count: {df_obs_numeric.duplicated(subset=subset_cols,keep=False).sum()}")

Duplicate observations count: 4420


In [209]:
# deduplicate observation df based on subset columns

deduped = df_obs_numeric.drop_duplicates(subset=subset_cols)
deduped.duplicated().sum()

0

In [211]:
# assign deduplicated df back to numeric observation df
df_obs_numeric = deduped

In [213]:
# rerun duplicates check after cleaning

find_duplicates(dfs_updated)

Patients
Duplicates Count: 0
Observations Numeric
Duplicates Count: 234
Encounters
Duplicates Count: 0
Conditions
Duplicates Count: 0


In [215]:
# validate deduplication

# dictionary of deduplicated dataframes
dfs_deduped =  {
    "Patients": df_patients,
    "Observations Numeric": df_obs_numeric,
    "Encounters": df_enc,
    "Conditions": df_cond
}

find_duplicates(dfs_deduped)

Patients
Duplicates Count: 0
Observations Numeric
Duplicates Count: 0
Encounters
Duplicates Count: 0
Conditions
Duplicates Count: 0


#### Semantic Checks

In [217]:
# check for kinds of labs present in observations df
# ideally want vitals, HbA1c, BMI, random glucose, BUN/Cr, filtration rates

df_obs_numeric['name'].value_counts().head(30)

name
Pain severity - 0-10 verbal numeric rating [Score] - Reported                                                    25379
Body Weight                                                                                                      15624
Respiratory rate                                                                                                 15239
Heart rate                                                                                                       15239
Body Height                                                                                                      14857
Body mass index (BMI) [Ratio]                                                                                    13773
Sodium [Moles/volume] in Blood                                                                                   10015
Creatinine [Mass/volume] in Blood                                                                                10015
Glucose [Mass/volume] in Blood             

In [219]:
# conditions
df_cond[(df_cond['is_prediabetes'] & df_cond['is_diabetes'])].head(10)

,patient_id,condition_id,code,name,onset,recorded_date,clinical_status,is_prediabetes,is_diabetes
10,a8a4c7d6-722a-a914-04b6-a23a7c3496e2,a8a4c7d6-722a-a914-fd83-023b0a892baf,714628002,Prediabetes (finding),2018-07-20 09:10:07+00:00,2018-07-20 09:10:07+00:00,active,True,True
92,7e0366e0-5cf1-1f51-f609-3ec9c2197140,7e0366e0-5cf1-1f51-fe6b-d07139361ab5,714628002,Prediabetes (finding),2020-05-29 04:33:52+00:00,2020-05-29 04:33:52+00:00,active,True,True
225,b75f55d6-a56b-611e-228a-8b5c87031609,b75f55d6-a56b-611e-5664-0f12f2d3f2cd,714628002,Prediabetes (finding),2011-12-02 20:47:36+00:00,2011-12-02 20:47:36+00:00,active,True,True
347,7959479b-5d1e-a9b6-bf6c-d09240490c0e,7959479b-5d1e-a9b6-0d44-1a4b89748b44,714628002,Prediabetes (finding),2025-04-29 07:15:40+00:00,2025-04-29 07:15:40+00:00,active,True,True
383,dbc13ff9-face-3a16-b7b2-0073d73dea9a,dbc13ff9-face-3a16-3595-9b1b323e77e9,714628002,Prediabetes (finding),1987-02-27 17:12:34+00:00,1987-02-27 17:12:34+00:00,active,True,True
665,3e5092da-7854-5cfe-2cfb-8f3c96e182fd,3e5092da-7854-5cfe-4033-d2939757e402,714628002,Prediabetes (finding),1971-05-02 19:00:43+00:00,1971-05-02 19:00:43+00:00,active,True,True
840,5025c250-3ac9-baa5-dd1e-62dbf6f69c2c,5025c250-3ac9-baa5-820c-5744fd6437d2,714628002,Prediabetes (finding),2009-09-30 16:49:27+00:00,2009-09-30 16:49:27+00:00,active,True,True
971,ff814ec6-8d8e-5389-f58e-15468c445d70,ff814ec6-8d8e-5389-61a2-7fac0b4593d8,714628002,Prediabetes (finding),1996-02-02 02:19:57+00:00,1996-02-02 02:19:57+00:00,active,True,True
1028,d6762ace-3c39-629e-42b7-de52c101adee,d6762ace-3c39-629e-0c45-c9e901aea578,714628002,Prediabetes (finding),2007-09-01 08:11:02+00:00,2007-09-01 08:11:02+00:00,active,True,True
1065,96523502-dee5-a97b-e441-7d040dec38bc,96523502-dee5-a97b-9285-aa381c44044e,714628002,Prediabetes (finding),1944-07-17 18:21:48+00:00,1944-07-17 18:21:48+00:00,active,True,True


In [221]:
df_enc['class'].value_counts().head(10)

class
AMB     61955
EMER     2320
IMP      1263
HH        510
VR        118
Name: count, dtype: int64

#### Dates Conversion Validation

In [223]:
df_patients['birth_date'] = pd.to_datetime(df_patients['birth_date'], utc=True)

In [225]:
df_patients.dtypes, df_obs_numeric.dtypes, df_enc.dtypes, df_cond.dtypes

(patient_id                 object
 gender                     object
 birth_date    datetime64[ns, UTC]
 dtype: object,
 patient_id                 object
 code                       object
 name                       object
 value                     float64
 unit                       object
 time          datetime64[ns, UTC]
 dtype: object,
 patient_id                   object
 encounter_id                 object
 start           datetime64[ns, UTC]
 end             datetime64[ns, UTC]
 class                        object
 dtype: object,
 patient_id                      object
 condition_id                    object
 code                             int64
 name                            object
 onset              datetime64[ns, UTC]
 recorded_date      datetime64[ns, UTC]
 clinical_status                 object
 is_prediabetes                    bool
 is_diabetes                       bool
 dtype: object)

## Cohort Segmentation for Modeling

#### Define Cohort
##### We are interested in the adult population so all patients must be 18 years or older.

In [227]:
df_patients.head(2)

,patient_id,gender,birth_date
0,a8a4c7d6-722a-a914-04b6-a23a7c3496e2,female,1999-05-21 00:00:00+00:00
1,779b82f4-c8cd-9764-2f80-28f0b176a418,female,2000-03-03 00:00:00+00:00


In [229]:
# add age column for patients
reference_date = pd.Timestamp.now(tz='UTC')
df_patients['age'] = (reference_date - df_patients['birth_date']).dt.days // 365
df_patients.head()

,patient_id,gender,birth_date,age
0,a8a4c7d6-722a-a914-04b6-a23a7c3496e2,female,1999-05-21 00:00:00+00:00,26
1,779b82f4-c8cd-9764-2f80-28f0b176a418,female,2000-03-03 00:00:00+00:00,25
2,e696153b-4bda-a741-04fe-a1fd43f60fd9,female,1972-04-14 00:00:00+00:00,53
3,7e0366e0-5cf1-1f51-f609-3ec9c2197140,female,1998-03-26 00:00:00+00:00,27
4,f286e228-2d19-4405-da1c-23dcae75169b,female,1971-04-03 00:00:00+00:00,54


In [231]:
# filter all patients >= 18 years

df_patients_adult = df_patients.loc[df_patients['age'] >= 18].copy()

In [235]:
# helper function to filter to adult patients
def filter_to_adult(df, adult_ids, id_col='patient_id'):
    """
    filter all patients in dataframe that are present in adult_ids
    """
    return df.loc[df[id_col].isin(adult_ids)].copy()

In [237]:
# filter all dataframes for adult patient only

adult_pt_ids = df_patients_adult['patient_id']

df_obs_adult = filter_to_adult(df_obs_numeric, adult_pt_ids)
df_enc_adult = filter_to_adult(df_enc, adult_pt_ids)
df_cond_adult = filter_to_adult(df_cond, adult_pt_ids)

In [239]:
# check dimensions after filtering

for df in [df_patients_adult,
          df_obs_adult,
          df_enc_adult,
          df_cond_adult]:
    print(df.shape)

(956, 4)
(427494, 6)
(61204, 5)
(39093, 9)


## Outcome: Diabetes onset
### Definition:
#### First occurrence of Type 2 Diabetes diagnosis OR
#### First occurrence of diabetes-related complication

### Cases Cohort

In [241]:
# Cases cohort: all patients with diabetes
# filter first date of diabetes diagnosis

df_dm_events = df_cond_adult.loc[df_cond_adult['is_diabetes']].copy()
df_dm_onset = (
    df_dm_events
    .groupby('patient_id')['onset']
    .min()
    .reset_index()
    .rename(columns={'onset':'diabetes_onset_date'})
)

In [243]:
print(
    f"\nDiabetes Cases df Shape: {df_dm_onset.shape}\n"
    f"\nFirst few rows:\n"
    f"\n{df_dm_onset.head()}")


Diabetes Cases df Shape: (503, 2)

First few rows:

                             patient_id       diabetes_onset_date
0  00a22b4e-fa50-0dd1-ac9e-f91e8b7fc2b4 1947-10-13 04:52:13+00:00
1  00b05b83-aee7-281a-fe2b-b1151bd7c5ee 2009-07-06 14:56:01+00:00
2  011dcab7-8543-84f8-5842-fa84972bbf25 2020-02-07 17:52:29+00:00
3  016329fd-8f76-971e-d716-709f9f529dc7 1997-06-08 11:06:54+00:00
4  01fac8f1-8eac-04bb-1d5c-8dbead000d6c 2002-06-09 16:06:33+00:00


### Controls Cohort

In [245]:
df_last_seen = (
    df_enc_adult
    .groupby('patient_id')['start']
    .max()
    .reset_index()
    .rename(columns={'start':'last_encounter_date'})
)

In [247]:
df_cohort = df_patients_adult[['patient_id']].copy()
df_cohort = df_cohort.merge(df_dm_onset, on='patient_id', how='left')
df_cohort = df_cohort.merge(df_last_seen, on='patient_id', how='left')

df_cohort.head()

,patient_id,diabetes_onset_date,last_encounter_date
0,a8a4c7d6-722a-a914-04b6-a23a7c3496e2,2018-07-20 09:10:07+00:00,2025-01-21 09:10:07+00:00
1,779b82f4-c8cd-9764-2f80-28f0b176a418,NaT,2025-05-24 03:32:57+00:00
2,e696153b-4bda-a741-04fe-a1fd43f60fd9,NaT,2022-06-10 14:14:34+00:00
3,7e0366e0-5cf1-1f51-f609-3ec9c2197140,2020-05-29 04:33:52+00:00,2025-10-31 04:33:52+00:00
4,f286e228-2d19-4405-da1c-23dcae75169b,2021-05-29 14:35:55+00:00,2025-10-11 14:35:55+00:00


In [249]:
labels = df_cohort['diabetes_onset_date'].notna().astype(int)
labels.head(2)

0    1
1    0
Name: diabetes_onset_date, dtype: int64

In [253]:
# create labels column
df_cohort['label'] = labels

# index_date will be the date we predict from, either 1 year prior
# from dm diagnosis for dm patients or
# 1 year prior to last encounter date for patients
# who never develop dm

df_cohort['index_date'] = np.where(
    df_cohort['label'] == 1,
    df_cohort['diabetes_onset_date'] - pd.Timedelta(days=365),
    df_cohort['last_encounter_date'] - pd.Timedelta(days=365)
)

df_cohort.head()


,patient_id,diabetes_onset_date,last_encounter_date,label,index_date
0,a8a4c7d6-722a-a914-04b6-a23a7c3496e2,2018-07-20 09:10:07+00:00,2025-01-21 09:10:07+00:00,1,2017-07-20 09:10:07+00:00
1,779b82f4-c8cd-9764-2f80-28f0b176a418,NaT,2025-05-24 03:32:57+00:00,0,2024-05-24 03:32:57+00:00
2,e696153b-4bda-a741-04fe-a1fd43f60fd9,NaT,2022-06-10 14:14:34+00:00,0,2021-06-10 14:14:34+00:00
3,7e0366e0-5cf1-1f51-f609-3ec9c2197140,2020-05-29 04:33:52+00:00,2025-10-31 04:33:52+00:00,1,2019-05-30 04:33:52+00:00
4,f286e228-2d19-4405-da1c-23dcae75169b,2021-05-29 14:35:55+00:00,2025-10-11 14:35:55+00:00,1,2020-05-29 14:35:55+00:00


In [259]:
# develop features dataframe
# merge observaitons df with cohort df on patient_id

df_obs_features = df_obs_adult.merge(
    df_cohort[['patient_id', 'index_date']],
    on='patient_id'
    )
df_obs_features.head()

,patient_id,code,name,value,unit,time,index_date
0,a8a4c7d6-722a-a914-04b6-a23a7c3496e2,6206-7,Peanut IgE Ab [Units/volume] in Serum,0.088943,kU/L,2016-06-26 14:58:45+00:00,2017-07-20 09:10:07+00:00
1,a8a4c7d6-722a-a914-04b6-a23a7c3496e2,6273-7,Walnut IgE Ab [Units/volume] in Serum,0.086704,kU/L,2016-06-26 14:58:45+00:00,2017-07-20 09:10:07+00:00
2,a8a4c7d6-722a-a914-04b6-a23a7c3496e2,6082-2,Codfish IgE Ab [Units/volume] in Serum,0.033234,kU/L,2016-06-26 14:58:45+00:00,2017-07-20 09:10:07+00:00
3,a8a4c7d6-722a-a914-04b6-a23a7c3496e2,6246-3,Shrimp IgE Ab [Units/volume] in Serum,0.298170,kU/L,2016-06-26 14:58:45+00:00,2017-07-20 09:10:07+00:00
4,a8a4c7d6-722a-a914-04b6-a23a7c3496e2,6276-0,Wheat IgE Ab [Units/volume] in Serum,0.151060,kU/L,2016-06-26 14:58:45+00:00,2017-07-20 09:10:07+00:00


In [265]:
# filter observations that are taken 1 year prior to the index_date for each patient
df_obs_features = df_obs_features[
    (df_obs_features['time'] < df_obs_features['index_date']) &
    (df_obs_features['time'] >= df_obs_features['index_date'] - pd.Timedelta(days=365))
]

In [271]:
df_obs_features.head()

,patient_id,code,name,value,unit,time,index_date
24,a8a4c7d6-722a-a914-04b6-a23a7c3496e2,8302-2,Body Height,155.800,cm,2017-06-23 09:10:07+00:00,2017-07-20 09:10:07+00:00
25,a8a4c7d6-722a-a914-04b6-a23a7c3496e2,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,1.000,{score},2017-06-23 09:10:07+00:00,2017-07-20 09:10:07+00:00
26,a8a4c7d6-722a-a914-04b6-a23a7c3496e2,29463-7,Body Weight,56.300,kg,2017-06-23 09:10:07+00:00,2017-07-20 09:10:07+00:00
27,a8a4c7d6-722a-a914-04b6-a23a7c3496e2,39156-5,Body mass index (BMI) [Ratio],23.200,kg/m2,2017-06-23 09:10:07+00:00,2017-07-20 09:10:07+00:00
28,a8a4c7d6-722a-a914-04b6-a23a7c3496e2,59576-9,Body mass index (BMI) [Percentile] Per age and...,70.058,%,2017-06-23 09:10:07+00:00,2017-07-20 09:10:07+00:00


In [277]:
# identify vounts of LOINC lab codes
df_obs_features[['code']].value_counts().head(20)


code   
72514-3    630
29463-7    455
9279-1     436
8867-4     436
8302-2     428
39156-5    421
55758-7    291
70274-6    214
718-7      150
6690-2     147
777-3      147
786-4      147
787-2      147
788-0      147
789-8      147
785-6      147
2571-8     143
4544-3     143
2085-9     143
2093-3     143
Name: count, dtype: int64

In [283]:
feature_codes = {
    '29463-7',   # weight
    '8302-2',    # height
    '39156-5',   # BMI
    '8867-4',    # heart rate
    '9279-1',    # respiratory rate
    '70274-6',   # MAP
    '2571-8',    # triglycerides
    '4544-3',    # HDL
    '2093-3',    # LDL
    '2085-9'     # total cholesterol
}

In [285]:
df_obs_features = df_obs_features[
    df_obs_features['code'].isin(feature_codes)
    ].copy()

In [297]:
df_obs_features.shape

(2962, 7)

In [299]:
df_obs_features.head()

,patient_id,code,name,value,unit,time,index_date
24,a8a4c7d6-722a-a914-04b6-a23a7c3496e2,8302-2,Body Height,155.8,cm,2017-06-23 09:10:07+00:00,2017-07-20 09:10:07+00:00
26,a8a4c7d6-722a-a914-04b6-a23a7c3496e2,29463-7,Body Weight,56.3,kg,2017-06-23 09:10:07+00:00,2017-07-20 09:10:07+00:00
27,a8a4c7d6-722a-a914-04b6-a23a7c3496e2,39156-5,Body mass index (BMI) [Ratio],23.2,kg/m2,2017-06-23 09:10:07+00:00,2017-07-20 09:10:07+00:00
29,a8a4c7d6-722a-a914-04b6-a23a7c3496e2,8867-4,Heart rate,73.0,/min,2017-06-23 09:10:07+00:00,2017-07-20 09:10:07+00:00
30,a8a4c7d6-722a-a914-04b6-a23a7c3496e2,9279-1,Respiratory rate,12.0,/min,2017-06-23 09:10:07+00:00,2017-07-20 09:10:07+00:00


In [295]:
df_obs_features.groupby('code')['patient_id'].nunique()

code
2085-9     113
2093-3     113
2571-8     113
29463-7    395
39156-5    393
4544-3      96
70274-6    209
8302-2     395
8867-4     396
9279-1     396
Name: patient_id, dtype: int64

In [304]:
df_cohort.head()

,patient_id,diabetes_onset_date,last_encounter_date,label,index_date
0,a8a4c7d6-722a-a914-04b6-a23a7c3496e2,2018-07-20 09:10:07+00:00,2025-01-21 09:10:07+00:00,1,2017-07-20 09:10:07+00:00
1,779b82f4-c8cd-9764-2f80-28f0b176a418,NaT,2025-05-24 03:32:57+00:00,0,2024-05-24 03:32:57+00:00
2,e696153b-4bda-a741-04fe-a1fd43f60fd9,NaT,2022-06-10 14:14:34+00:00,0,2021-06-10 14:14:34+00:00
3,7e0366e0-5cf1-1f51-f609-3ec9c2197140,2020-05-29 04:33:52+00:00,2025-10-31 04:33:52+00:00,1,2019-05-30 04:33:52+00:00
4,f286e228-2d19-4405-da1c-23dcae75169b,2021-05-29 14:35:55+00:00,2025-10-11 14:35:55+00:00,1,2020-05-29 14:35:55+00:00


In [306]:
# save df_cohort and df_obs_features to csv and transiiton to feature engineering
df_cohort.to_csv(
    '/Users/sanasiddiqui/Desktop/Current Desktop/Data Portfolio Projects/Diabetes_Onset_Prediction_FHIR/Data_Diabetes_Prediction/Cleaned/df_cohort.csv',
    index=False
),
df_obs_features.to_csv(
    '/Users/sanasiddiqui/Desktop/Current Desktop/Data Portfolio Projects/Diabetes_Onset_Prediction_FHIR/Data_Diabetes_Prediction/Cleaned/df_obs_features.csv',
    index=False
)